In [3]:
import pandas as pd
import numpy as np
##读取数据
df=pd.read_csv('boston_housing_data.csv')
##数据预处理
print(df.info())  ##数据的可视化，发现MEDV有空数据，需要剔除，我们把这些空数据作为测试集来预测

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    int64  
 9   TAX      506 non-null    float64
 10  PIRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     452 non-null    float64
dtypes: float64(13), int64(1)
memory usage: 55.5 KB
None


In [19]:
text=df[df['MEDV'].isnull()]
##将测试集存储到文件中
text.to_csv("波士顿房价测试集.csv",index=False,sep=',')
##然后剔除原数据的空值，然后为了保留上述的过程，所以我没有inplace=TRUE
df1=df.dropna()

##### 对整体数据及集的分析
我们不难发现的是，有些数据的大小的相差过大，有超过300的，也有小于1的，这样的话不利于进行线性回归，也就是说我们要进行数据的处理，把他们变小一点，不过这一步可以不去做，因为没有太大的必要，差距再可控范围内。

In [5]:
##回归直线拟合--正规方程法
def Linear_fitting(X,Y):
    theta=np.linalg.inv(X.T*X)*X.T*Y
    return theta
##数据处理--转化为矩阵
date=np.matrix(df1)
X=date[:,0:13]
Y=date[:,13]
X=np.c_[np.ones(Y.size),X]
theta=Linear_fitting(X,Y)
theta


matrix([[ 2.08663654e+01],
        [-2.38024776e-01],
        [ 3.82205042e-02],
        [ 5.13558367e-02],
        [ 2.43504780e+00],
        [-1.16579857e+01],
        [ 5.11015834e+00],
        [-6.09415860e-03],
        [-1.27151396e+00],
        [ 2.94444280e-01],
        [-1.13598117e-02],
        [-8.31030389e-01],
        [ 1.23142862e-02],
        [-5.20753302e-01]])

In [6]:
##代价函数，测试拟合效果
def Cost_funtion(X,Y,theta):
    cost=np.power(((X*theta)-Y),2)
    return np.sum(cost)/(2*len(X))
cost=Cost_funtion(X,Y,theta)
cost


10.044277819270544

 从上述的代价可以看出，本次拟合效果相对来说还是比较好的，符合要求，这个时候我们就可以用此次模型来对缺失的数据进行填充

In [7]:
text=text.drop('MEDV',axis=1)
date_text=np.matrix(text)
X_text=date_text[:,0:13]
X_text=np.c_[np.ones(X_text.shape[0]),X_text]
Y1=X_text*theta
Y1

matrix([[533.34132286],
        [305.7592669 ],
        [314.7522323 ],
        [266.08603437],
        [279.16885241],
        [269.39089315],
        [294.75251892],
        [241.88836879],
        [283.39949055],
        [361.99350524],
        [295.189447  ],
        [310.43509737],
        [254.38116499],
        [327.88948149],
        [291.01069604],
        [267.64211479],
        [303.97245818],
        [297.38699675],
        [292.32421101],
        [282.89122957],
        [282.63662966],
        [311.11199355],
        [337.71123046],
        [343.79043161],
        [456.3616027 ],
        [622.4106857 ],
        [583.94201915],
        [612.59556695],
        [455.71170996],
        [570.37808008],
        [598.52554267],
        [551.97908159],
        [471.10894107],
        [612.12471094],
        [455.66497075],
        [351.5232114 ],
        [317.25609246],
        [596.92340564],
        [399.74755687],
        [497.28352318],
        [515.86264646],
        [512.194

上述就是用正规方程法求出来的结果，接下来我继续用梯度下降法来解决这个问题

In [61]:
##要完成梯度下降法，那么我们先是要进行归一化，从而方便我们快速的迭代出我们需要的数据
df2=df1
train=[]
for x in df2.columns.tolist()[0:13]:
    s=np.sum(df2[x])
    xs=df2[x]/s
    train.append(xs)
##构建数据--转化为矩阵并且计算
datax=np.matrix(train)
dataxT=datax.T
datay=np.matrix(df2)
X_gd=dataxT[:,:13]
Y_gd=datay[:,13]
X_gd=np.c_[np.ones(Y_gd.size),X_gd]
l=X_gd.shape[1]
theta_gd=np.zeros(l)
theta_gd=np.matrix(theta_gd)
theta_gd=theta_gd.T
##梯度下降法
def Gradient_descent(X,Y,theta,a,num_lisers):
    m=Y.size
    for i in range(0,num_lisers):
        y_hat=np.dot(X[:,1:],theta[1:])+theta[0]
        dy=Y-y_hat
        theta[0] += a*np.sum(dy)
        theta[1:] += a*np.dot(X[:,1:].T,dy)
    return theta
##计算并获取数据
a=0.001
num_lisers=5000
theta_gd=Gradient_descent(X_gd,Y_gd,theta_gd,a,num_lisers)
theta_gd

matrix([[ 23.78821169],
        [-21.3753154 ],
        [ 27.15648116],
        [-11.65912903],
        [ 22.13905409],
        [ -2.95510022],
        [  3.44956767],
        [ -5.4726575 ],
        [  3.0400322 ],
        [ -8.84613101],
        [ -5.89750501],
        [ -2.35814797],
        [  2.17380719],
        [-16.47472291]])

In [12]:
##根据上述的结果进行代价运算
cost_gd=Cost_funtion(X_gd,Y_gd,theta_gd)
cost_gd

38.72065272945117

 emmmmm,很明显，这次梯度下降的效果就差了很多，不过相对来说，应该问题不是很大，也是差不多的，然后这次的结果我们也可以来计算测试集的值

In [13]:
Y2=X_text*theta_gd
Y2

matrix([[62.53890645],
        [76.07606688],
        [76.22201535],
        [74.86907065],
        [73.52166794],
        [75.28281221],
        [75.20191734],
        [74.64349671],
        [75.28666285],
        [70.0315046 ],
        [75.66652605],
        [75.65951535],
        [75.2140014 ],
        [74.6815855 ],
        [75.34940198],
        [74.76461843],
        [75.74377   ],
        [75.56349246],
        [75.23605238],
        [74.94247612],
        [71.59507126],
        [74.76006817],
        [74.11224513],
        [72.00473338],
        [65.3464452 ],
        [56.28403281],
        [58.51388698],
        [58.88971151],
        [66.59951696],
        [61.66404074],
        [58.45316299],
        [57.47177412],
        [62.338046  ],
        [57.46919593],
        [57.71975334],
        [71.43989479],
        [69.39477431],
        [56.99264722],
        [55.17767663],
        [56.00537845],
        [60.23286812],
        [60.87392737],
        [61.78670633],
        [57

有这两种数据我们可以看到，之前作为测试集的数据，也就是我们剔除的数据，其实是不是很合理的，比较两个结果可以发现按道理来说，不应该是这样的，所以，咳咳，应该不能把这些数据看出测试集，没有参考型，不过可以通过代价函数来看出拟合的效果。而且根据数据的整体情况来看的话，会发现缺失的数据有很大的问题，所以本次的测试集其实是错误的。
不过因为本次两种方法得到的模型其实问题比较大，所以接下来继续优化梯度下降法

In [62]:
##优化梯度算法
def Gradient_descent1(X,Y,theta,a,num_lisers):
    m=Y.size
    for i in range(0,num_lisers):
        theta=theta-(a/m)*X.T*(X*theta-Y)
    return theta
##对数据进行标准化处理
df3=df1
train1=[]
for x in df3.columns.tolist()[0:13]:
    s=np.sum(df2[x])
    m=np.mean(df[x])
    ms=np.ones(Y.size)*m
    xs1=(df3[x]-ms)/s
    train1.append(xs1)
datax1=np.matrix(train1)
dataxT1=datax1.T
datay1=np.matrix(df3)
X_gd1=dataxT1[:,:13]
Y_gd1=datay1[:,13]
X_gd1=np.c_[np.ones(Y_gd1.size),X_gd1]
l=X_gd1.shape[1]
theta_gd1=np.zeros(l)
theta_gd1=np.matrix(theta_gd1)
theta_gd1=theta_gd1.T
a=0.01
num_lisers=5000
theta_gd1=Gradient_descent1(X_gd1,Y_gd1,theta_gd1,a,num_lisers)
theta_gd1

matrix([[ 2.37376096e+01],
        [-4.82768881e-01],
        [ 6.13643444e-01],
        [-2.58403178e-01],
        [ 4.75255730e-01],
        [-1.22457336e-01],
        [-1.65911498e-03],
        [-1.19556102e-01],
        [ 4.20008937e-02],
        [-6.75544604e-01],
        [-1.29417546e-01],
        [-1.24177163e-01],
        [ 5.29574236e-02],
        [-3.69800280e-01]])

In [64]:
##代价估计
cost_gd1=Cost_funtion(X_gd1,Y_gd1,theta_gd1)
cost_gd1

38.68479001515334

发现结果并没有变化，已经麻了，不过我觉得问题还是有，在初始化设置theta那里，不过那些都是比较后面的内容，就先不做太多的阐述了